# Requirements

In [4]:
!pip3 install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.5/469.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.5/437.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 12.2 MB/s eta 0:00:00
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9611 sha256=30c5cec6bed043c8570d7165d3328c89d563083c5a3dde5ba023375e4b9188cc
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
  Created wheel for mmcv: filename=mmcv-2.0.1-cp310-cp310-linux_x86_64.whl size=37092657 sha256=7cb0e5f629227b09a77dc9b20dafadc94313fff1d17cb1f69715b9a09f79bd57
  Stored in directory: /root/.cache/pip/wheels/17/58/eb/d8f23c5eb52bc7441f5b41a682a03b35277fc8cd4779d200b9
Successfully built progress mmcv
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstallin

In [5]:
!pip install mmcv-full

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv-full: filename=mmcv_full-1.7.1-cp310-cp310-linux_x86_64.whl size=30392963 sha256=c10f677fbefa9609e41d01e969c9ced50269596dc500ec5efe1eede9289e5556
  Stored in directory: /root/.cache/pip/wheels/47/9a/65/470be18e21a8f2d085a024f0731508273543de0b5f79d9ddd4
Successfully built mmcv-full


# Train Yolox

In [ ]:
!unrar Data.rar

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: labels/train/f4b1c975a462deb3.txt  
  inflating: labels/train/4bcba12d3acd5f78.txt  
  inflating: labels/train/93bc5f9bee726d74.txt  
  inflating: labels/train/cef407c5495b7ec9.txt  
  inflating: labels/train/32a8a4b72e4010e5.txt  
  inflating: labels/train/cf30e382cf1bb470.txt  
  inflating: labels/train/0faa88806c46de26.txt  
  inflating: labels/train/e5615f4588d05951.txt  
  inflating: labels/train/085e7d6eb4dcf064.txt  
  inflating: labels/train/fa2d4ede6cc40918.txt  
  inflating: labels/train/d568284781710f93.txt  
  inflating: labels/train/f7273635becb5f69.txt  
  inflating: labels/train/d906610476b2ce60.txt  
  inflating: labels/train/243ed87b0ccd6bb0.txt  
  inflating: labels/train/1e4c0c02f19feaf2.txt  
  inflating: labels/train/71e464acb818a95f.txt  
  inflating: labels/train/e0733e1ee4fcdc50.txt  
  inflating: labels/train/09d39893f8565488.txt  
  inflating: labels/train/056510d1759b034c.txt  
  inflating:

In [ ]:
!python train.py gpus='0' backbone="CSPDarknet-s" num_epochs=3 exp_id="coco_CSPDarknet-s_640x640" use_amp=True val_intervals=2 data_num_workers=6 batch_size=4 load_model="pretrained_models/yolox-s.pth"

# Predict

The result(both labels and images) will be store in output folder

In [18]:
!python yolox/predict.py gpus='0' backbone="CSPDarknet-s" vis_thresh=0.25 load_model="pretrained_models/model_best.pth" img_dir='Data/images/test'

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
save image to output/test/render_7635.jpg
 11% 87/801 [00:06<00:38, 18.62it/s]------------------------------
87/801, /content/Data/images/test/render_1550.jpg
[pre_process] time 0.021754980087280273
[inference] batch=1x3x640x640 time: 0.01583695411682129s
[post_process] time 0.0022118091583251953
[pre_process + inference + post_process] time cost: 0.04142189025878906s
save image to output/test/render_1550.jpg
------------------------------
88/801, /content/Data/images/test/render_5767.jpg
[pre_process] time 0.021025657653808594
[inference] batch=1x3x640x640 time: 0.015814542770385742s
[post_process] time 0.0019767284393310547
[pre_process + inference + post_process] time cost: 0.04030108451843262s
save image to output/test/render_5767.jpg
 11% 89/801 [00:06<00:38, 18.73it/s]------------------------------
89/801, /content/Data/images/test/render_3138.jpg
[pre_process] time 0.021470069885253906
[inference] batch=1x3x640x640 time: 0

# Metrics

Calculating iou, confident and draw charts

In [ ]:
import os

def calculate_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) between two bounding boxes.

    Args:
        box1 (list): [x_center, y_center, width, height] of first box
        box2 (list): [x_center, y_center, width, height] of second box

    Returns:
        float: Intersection over Union (IoU) value between the two boxes
    """
    # Calculate the coordinates of the intersection rectangle
    x1 = max(box1[0] - box1[2] / 2, box2[0] - box2[2] / 2)
    y1 = max(box1[1] - box1[3] / 2, box2[1] - box2[3] / 2)
    x2 = min(box1[0] + box1[2] / 2, box2[0] + box2[2] / 2)
    y2 = min(box1[1] + box1[3] / 2, box2[1] + box2[3] / 2)

    # Calculate the area of the intersection rectangle
    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    # Calculate the area of the union of the two bounding boxes
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]
    union_area = box1_area + box2_area - intersection_area

    # Calculate the IoU value
    iou = intersection_area / union_area

    return iou

def calculate_average(data_list):
    sum = 0
    for i in data_list:
      sum += i
    return sum / len(data_list)

def str_to_number(str_list):
  temp = []
  for i in str_list:
    temp.append(float(i))
  return temp


real_labels = os.listdir("Data/labels/test")
result_labels = os.listdir("output/labels")

iou_dic = {}
real_labels_dic = {}
result_labels_dic = {}
conf_dic = {}

for real in real_labels:
  with open("Data/labels/test/" + real) as real_file:
    real_temp = []
    for line in real_file:
      real_temp.append(str_to_number(line.split(" ")[1:]))
  real_labels_dic[real] = real_temp

for result in result_labels:
  with open("output/labels/" + result) as result_file:
    result_temp = []
    conf_temp = []
    for line in result_file:
      result_temp.append(str_to_number(line.split(" ")[1:]))
      conf_temp.append(float(line.split(" ")[0]))
    result_labels_dic[result] = result_temp
    conf_dic[result] = conf_temp

for key in result_labels_dic.keys():
  iou_list = []
  conf_list = []
  for result_line_index in range(len(result_labels_dic[key])):
    iou = 0
    conf = 0
    for real_line in real_labels_dic[key]:

      if iou < calculate_iou(result_labels_dic[key][result_line_index], real_line):
        iou = calculate_iou(result_labels_dic[key][result_line_index], real_line)
    conf = conf_dic[key][result_line_index]
    iou_list.append(iou)
    conf_list.append(conf)
  conf_dic[key] = calculate_average(conf_list)
  iou_dic[key] = calculate_average(iou_list)


In [ ]:
sum = 0
for key in iou_dic.keys():
  sum += iou_dic[key]
print(sum/len(iou_dic.keys()))

In [ ]:
sum = 0
for key in conf_dic.keys():
  sum += conf_dic[key]
print(sum/len(conf_dic.keys()))

In [ ]:
import matplotlib.pyplot as plt

x_values, y_values = [key for key in iou_dic.keys()], list(iou_dic.values())

plt.bar([i for i in range(50)], y_values[0:50])
plt.xlabel("image_number")
plt.ylabel("IoU")
plt.title("IoU distribution of first 50 images")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

x_values, y_values = [key for key in conf_dic.keys()], list(conf_dic.values())

plt.bar([i for i in range(50)], y_values[0:50])
plt.xlabel("image_number")
plt.ylabel("confident")
plt.title("Confident distribution of first 50 images")
plt.show()

In [ ]:
import matplotlib.pyplot as plt


s1 = 0
s2 = 0
s3 = 0
s4 = 0

for key in iou_dic.keys():
  value = iou_dic[key]
  if value >= 0 and value < 0.25:
    s1 += 1
  elif value >= 0.25 and value < 0.5:
    s2 += 1
  elif value >= 0.5 and value < 0.75:
    s3 += 1

print(s1, s2, s3, s4)

s1 = s1/len(iou_dic.keys())*100
s2 = s2/len(iou_dic.keys())*100
s3 = s3/len(iou_dic.keys())*100
s4 = 100 - s1 - s2 - s3

print(s1, s2, s3, s4)

labels = ["0-0.25", "0.26-0.5", "0.51-0.75", "0.76-1"]
sizes = [s1, s2, s3, s4]

# Create a pie chart
plt.pie(sizes, labels=[str(round(s1, 2))+"%", "", "", str(round(s4, 2))+"%"])

# Add title
plt.title("IoU Value Distribution")
plt.legend(labels, loc="lower right")

# Display the chart
plt.show()

In [ ]:
import matplotlib.pyplot as plt


s1 = 0
s2 = 0
s3 = 0
s4 = 0

for key in conf_dic.keys():
  value = conf_dic[key]
  if value >= 0 and value < 0.25:
    s1 += 1
  elif value >= 0.25 and value < 0.5:
    s2 += 1
  elif value >= 0.5 and value < 0.75:
    s3 += 1

print(s1, s2, s3, s4)
s1 = s1/len(conf_dic.keys())*100
s2 = s2/len(conf_dic.keys())*100
s3 = s3/len(conf_dic.keys())*100
s4 = 100 - s1 - s2 - s3

print(s1, s2, s3, s4)

labels = ["0-0.25", "0.26-0.5", "0.51-0.75", "0.76-1"]
sizes = [s1, s2, s3, s4]

# Create a pie chart
plt.pie(sizes, labels=["", str(round(s2, 2))+"%", str(round(s3, 2))+"%", str(round(s4, 2))+"%"])

# Add title
plt.title("Confident Value Distribution\n(with threshold of 0.25)")
plt.legend(labels, loc="lower right")

# Display the chart
plt.show()

In [ ]:
import statistics

# Example data
data = conf_dic.values()

# Calculate the mean
mean = statistics.mean(data)

# Calculate the median
median = statistics.median(data)

# Calculate the mode (returns a list of modes)
mode = statistics.mode(data)

# Calculate the standard deviation
stdev = statistics.stdev(data)

# Print the results
print("Mean:", mean)
print("Median:", median)
print("Mode:", mode)
print("Standard deviation:", stdev)

In [ ]:
import statistics

# Example data
data = iou_dic.values()

# Calculate the mean
mean = statistics.mean(data)

# Calculate the median
median = statistics.median(data)

# Calculate the mode (returns a list of modes)
mode = statistics.mode(data)

# Calculate the standard deviation
stdev = statistics.stdev(data)

# Print the results
print("Mean:", mean)
print("Median:", median)
print("Mode:", mode)
print("Standard deviation:", stdev)